# Init

In [1]:
import sys
import os
import json
import time

from tqdm import tqdm

cwd = os.getcwd()
os.chdir(cwd)
sys.path.append('tools')

import chat
import parse_data

In [2]:
system_prompt = ""

prompt_dir = os.path.join(cwd, 'prompt')
fname = 'table_finder-stage_1-v2.3.0.md'
prompt_fpath = os.path.join(prompt_dir, fname)

with open(prompt_fpath, 'r') as f:
    prompt_template = ''.join(f.readlines())

def make_prompt(query: str) -> str:

    prompt = prompt_template + query

    return prompt

In [3]:
question_path = os.path.join(cwd, 'data' + os.sep + 'question.json')

questions = parse_data.read_json(question_path)

# GLM-4-Plus

## Test

In [17]:
query = make_prompt(questions[12]['team'][0]['question'])

history = []

start_time = time.time()
message = chat.create_message(query, history=history, system_prompt=system_prompt, temperature=0.7, top_p=0.9, response_format='text')
end_time = time.time()

execution_time = end_time - start_time
usage = chat.get_token_usage(message, True)
content = chat.get_content(message, True)
history = chat.build_history(history, message=message)

{'prompt_tokens': 9455, 'completion_tokens': 391, 'total_tokens': 9846}
```json
{
    "raw_question": "今天是2020年10月27日，当日收盘价第3高的港股是？(以下都回答简称)",
    "data_source_reasoning": "这个问题询问的是2020年10月27日收盘价第3高的港股的简称。我们可以这样逐步分析： 1. **时间范围：** 问题指定了一个特定日期（2020年10月27日），因此需要查询该日期的股票行情数据。 2. **条件：** 问题关注的是'收盘价第3高'的公司，这意味着需要比较该日期所有港股的收盘价。 3. **数据类型：** 收盘价是股票行情数据的一部分，属于技术指标范畴。 4. **定位数据：** 查看数据库架构，'港股数据库' (HKStockDB) 数据库包含与港股行情相关的表格。在这个数据库下，'港股行情表现' (CS_HKStockPerformance) 表包含每个交易日收盘行情数据，包括收盘价等指标，因此适合用于筛选收盘价第3高的公司。 5. **排除不相关表：** 其他表如 '港股公司员工数量变动表' (HK_EmployeeChange) 或 '港股公司概况' (HK_StockArchives) 主要关注公司基本信息或员工数量，不包含收盘价信息，因此不适用于此查询。 6. **结论：** 因此，要回答这个问题，我们需要查询 HKStockDB 数据库中的 CS_HKStockPerformance 表，筛选出2020年10月27日的数据，并找出收盘价第3高的港股的简称。",
    "data_source": [
        {"question": "2020年10月27日收盘价第3高的港股是？", "database": "HKStockDB", "table": "CS_HKStockPerformance"}
    ],
    "database_dependency": true
}
```


In [18]:
t = questions[12]
t['table_finder'] = {}
t['table_finder']['stage_1']= [json.loads(content.strip('`json'))]
t['token_usage'] = {}
t['token_usage']['table_finder-stage_1'] = [usage]
t['time_usage'] = {}
t['time_usage']['table_finder-stage_1'] = [f"{execution_time:.2f}s"]
t = [t]

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + 'stage_1-glm_4_plus-table_finder-test.json')
parse_data.write_json(t, saved_path)

## ALL

In [21]:
answers = []

for question in tqdm(questions[:]):
    # the first question
    query = make_prompt(question['team'][0]['question'])

    history = []

    start_time = time.time()
    message = chat.create_message(query, history=history, system_prompt=system_prompt, temperature=0.7, top_p=0.9, response_format='text')
    end_time = time.time()

    execution_time = end_time - start_time
    usage = chat.get_token_usage(message, False)
    content = chat.get_content(message, False)

    res = question
    res['table_finder'] = {}
    res['table_finder']['stage_1']= [json.loads(content.strip('`json'))]
    res['token_usage'] = {}
    res['token_usage']['table_finder-stage_1'] = [usage]
    res['time_usage'] = {}
    res['time_usage']['table_finder-stage_1'] = [f"{execution_time:.2f}s"]

    answers.append(res)

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + 'stage_1-glm_4_plus-table_finder.json')
parse_data.write_json(answers, saved_path)

100%|██████████| 101/101 [37:36<00:00, 22.34s/it]


# Deepseek-chat

In [22]:
model = 'deepseek_v3'

In [23]:
deepseek_api = 'sk-ba0f5eed3bea4fa6be16eb33b139c684'

## Test

In [24]:
from openai import OpenAI

query = make_prompt(questions[12]['team'][0]['question'])

client = OpenAI(api_key= deepseek_api, base_url="https://api.deepseek.com")

start_time = time.time()
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "user", "content": query},
    ],
    stream=False,
    top_p=0.7,
    temperature=0.9
)
end_time = time.time()

response = json.loads(response.to_json())
content = response['choices'][0]['message']['content']

content = content.strip('`json')
usage = response['usage']
execution_time = end_time - start_time

In [25]:
t = questions[12]
t['table_finder'] = {}
t['table_finder']['stage_1']= [json.loads(content.strip('`json'))]
t['token_usage'] = {}
t['token_usage']['table_finder-stage_1'] = [usage]
t['time_usage'] = {}
t['time_usage']['table_finder-stage_1'] = [f"{execution_time:.2f}s"]
t = [t]

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'stage_1-{model}-table_finder-test.json')
parse_data.write_json(t, saved_path)

## ALL

In [26]:
answers = []

for question in tqdm(questions[:]):
    
    query = make_prompt(question['team'][0]['question'])

    start_time = time.time()
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "user", "content": query},
        ],
        stream=False,
        top_p=0.7,
        temperature=0.9
    )
    end_time = time.time()

    response = json.loads(response.to_json())
    content = response['choices'][0]['message']['content']

    content = content.strip('`json')
    usage = response['usage']
    execution_time = end_time - start_time

    res = question
    res['table_finder'] = {}
    res['table_finder']['stage_1']= [json.loads(content.strip('`json'))]
    res['token_usage'] = {}
    res['token_usage']['table_finder-stage_1'] = [usage]
    res['time_usage'] = {}
    res['time_usage']['table_finder-stage_1'] = [f"{execution_time:.2f}s"]

    answers.append(res)

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'stage_1-{model}-table_finder.json')
parse_data.write_json(answers, saved_path)

100%|██████████| 101/101 [12:01<00:00,  7.14s/it]


# Compare Results

Compare the stage 1 results => find the differences => get the correct answer.

In [86]:
import os
import json
import copy

# Path to the folder containing model answer files
dir_path = os.path.join(cwd, 'answer_tmp')

# List of model names
models = ['deepseek_v3', 'glm_4_plus']

# Create a dictionary of file paths for each model's JSON file
model_files = {model: os.path.join(dir_path, f"stage_1-{model}-table_finder.json") for model in models}

# Dictionary to store the data of each model
model_data = {}

# Read the JSON data for each model
for model, file_path in model_files.items():
    with open(file_path, 'r', encoding='utf-8') as f:
        model_data[model] = json.load(f)

# Dictionary to store the data_source for each question id across different models
data_sources = {}

# Traverse through each model's data to extract the data_source for each question id
for model in models:
    for entry in copy.deepcopy(model_data[model]):
        tmp = entry['table_finder']['stage_1'][0]['data_source']
        for i in range(len(tmp)):
            del tmp[i]['question']
        data_sources.setdefault(entry['tid'].replace(' ', ''), {}).update({model: tmp})

# Compare the data_source for each question id across models
for question_id, sources in data_sources.items():
    # Check if the data_source is consistent across models
    # Convert each model's data_source to a JSON string (to handle the dictionary comparison)
    serialized_sources = {model: json.dumps(ds, sort_keys=True, ensure_ascii=False) for model, ds in sources.items()}
    
    # If there are any differences in data_source, output the details
    if len(set(serialized_sources.values())) > 1:
        print(f"Question ID: {question_id}")

        q_id = int(question_id.split('-')[-1]) - 1
        
        # Calculate the maximum length of model names to align the output
        max_model_length = max(len(model) for model in models)
        # Print the data_source for each model, with aligned output
        for model in models:
            # Left-align model names with the calculated maximum length
            print(f"{model.ljust(max_model_length)}:")
            print('```')
            print(json.dumps(model_data[model][q_id]['table_finder']['stage_1'], indent=2, ensure_ascii=False))
            print('```')
        
        print("")

Question ID: tttt----4
deepseek_v3:
```
[
  {
    "raw_question": "互联网金融属于科技概念的什么分支？这个概念的英文名称是什么？",
    "data_source_reasoning": "这个问题询问的是互联网金融在科技概念中的分支以及其英文名称。我们可以这样逐步分析： 1. **问题类型：** 这是一个关于概念分类和英文名称的问题，涉及互联网金融在科技概念中的定位。 2. **数据相关性：** 提供的数据库架构中，'上市公司行业板块' (AStockIndustryDB) 数据库包含与行业和概念相关的表格，特别是 '概念板块常量表' (LC_ConceptList) 表，记录了A股市场中热点概念的相关信息，包括概念的分类和英文名称。 3. **定位数据：** '概念板块常量表' (LC_ConceptList) 表很可能包含互联网金融在科技概念中的分支信息以及其英文名称。 4. **排除不相关表：** 其他表如 '公司行业划分表' (LC_ExgIndustry) 或 '行业估值指标' (LC_IndustryValuation) 主要关注行业划分和估值指标，不涉及概念分类和英文名称，因此不适用于此查询。 5. **结论：** 因此，要回答这个问题，我们需要查询 AStockIndustryDB 数据库中的 LC_ConceptList 表，获取互联网金融在科技概念中的分支信息以及其英文名称。",
    "data_source": [
      {
        "question": "互联网金融属于科技概念的什么分支？这个概念的英文名称是什么？",
        "database": "AStockIndustryDB",
        "table": "LC_ConceptList"
      }
    ],
    "database_dependency": true
  }
]
```
glm_4_plus :
```
[
  {
    "raw_question": "互联网金融属于科技概念的什么分支？这个概念的英文名称是什么？",
    "data_source_reasoning": "这个问题询问的是互联网金融所属的科技概念分支以及其英文名称。我们可